### fine tuningしてそのモデルを利用してSPARQLクエリを生成する

In [ ]:
import os
import json
from openai import OpenAI
from functions.results_evaluater import evaluate_jaccard

db = 'rhea'

endpoint = os.environ[f"ENDPOINT_{db.upper()}"]

In [ ]:
client = OpenAI()

client.files.create(
  file=open(f"dataset/train_data_random_{db}.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-8TxJAq65iewX5dKwsiMFCZ', bytes=61329, created_at=1749983449, filename='train_data_random_3_rhea.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-8TxJAq65iewX5dKwsiMFCZ", 
  model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-u4bszZeAjf7nDhsRfsvwlnHL', created_at=1749983457, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-lp3fs8tdFXsNwFTg0uNKfSoD', result_files=[], seed=392625965, status='validating_files', trained_tokens=None, training_file='file-8TxJAq65iewX5dKwsiMFCZ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [68]:
client.fine_tuning.jobs.retrieve("ftjob-u4bszZeAjf7nDhsRfsvwlnHL")

FineTuningJob(id='ftjob-u4bszZeAjf7nDhsRfsvwlnHL', created_at=1749983457, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-lp3fs8tdFXsNwFTg0uNKfSoD', result_files=[], seed=392625965, status='validating_files', trained_tokens=None, training_file='file-8TxJAq65iewX5dKwsiMFCZ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 'auto', 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}})

In [69]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=3)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-u4bszZeAjf7nDhsRfsvwlnHL', created_at=1749983457, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-lp3fs8tdFXsNwFTg0uNKfSoD', result_files=[], seed=392625965, status='validating_files', trained_tokens=None, training_file='file-8TxJAq65iewX5dKwsiMFCZ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 'auto', 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}}), FineTuningJob(id='ftjob-SHttHTkKVjKfsGMuFXFR23vP', created_at=1749975847, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mi

In [ ]:
# save path
save_path = f'data/questions/easy_question_augmented_{db}_baseline_FT.json'
save_path_with_results = f'data/questions/easy_question_augmented_with_results_{db}_baseline_FT.json'

In [ ]:
with open(f"questions/json_format/{db}.json", 'r') as f:
	questions = json.load(f)

In [ ]:
import time
from functions.SPARQL_executer import execute_query


query_results = []
for question in questions:
    time.sleep(0.1)
    query_results += [execute_query(question, endpoint, "sparql", 10000, "")]
    print(question["id"], len(query_results[-1][0]))

# 各質問に対する結果を追加
for result, question_id in query_results:
    # 質問を見つけて結果を追加
    for question in questions:
        if question["id"] == question_id:
            question["results"] = result

with open(save_path_with_results, 'w') as f:
    json.dump(questions, f, indent=4)

Q1-1-1 1
Q1-2-1 1
Q1-3-1 1
Q1-4-1 1
Q1-5-1 1
Q1-6-1 1
Q2-1-1 3
Q2-2-1 3
Q2-3-1 3
Q2-4-1 3
Q2-5-1 3
Q2-6-1 3
Q3-1-1 3
Q3-2-1 3
Q3-3-1 3
Q3-4-1 3
Q3-5-1 3
Q3-6-1 3
Q3-1-2 9
Q3-2-2 9
Q3-3-2 9
Q3-4-2 9
Q3-5-2 9
Q3-6-2 9
Q4-1-1 1
Q4-2-1 1
Q4-3-1 1
Q4-4-1 1
Q4-5-1 1
Q4-6-1 1
Q5-1-1 1
Q5-2-1 1
Q5-3-1 1
Q5-4-1 1
Q5-5-1 1
Q5-6-1 1
Q6-1-1 1
Q6-2-1 1
Q6-3-1 1
Q6-4-1 1
Q6-5-1 1
Q6-6-1 1
Q7-1-1 1
Q7-2-1 1
Q7-3-1 1
Q7-4-1 1
Q7-5-1 1
Q7-6-1 1
Q8-1-1 1
Q8-2-1 1
Q8-3-1 1
Q8-4-1 1
Q8-5-1 1
Q8-6-1 1
Q9-1-1 20
Q9-2-1 20
Q9-3-1 20
Q9-4-1 20
Q9-5-1 20
Q9-6-1 20
Q10-1-1 1
Q10-2-1 1
Q10-3-1 1
Q10-4-1 1
Q10-5-1 1
Q10-6-1 1
Q11-1-1 10
Q11-2-1 10
Q11-3-1 10
Q11-4-1 10
Q11-5-1 10
Q11-6-1 10
Q12-1-1 5
Q12-2-1 5
Q12-3-1 5
Q12-4-1 5
Q12-5-1 5
Q12-6-1 5
Q13-1-1 1
Q13-2-1 1
Q13-3-1 1
Q13-4-1 1
Q13-5-1 1
Q13-6-1 1
Q14-1-1 1
Q14-2-1 1
Q14-3-1 1
Q14-4-1 1
Q14-5-1 1
Q14-6-1 1
Q15-1-1 2
Q15-2-1 2
Q15-3-1 2
Q15-4-1 2
Q15-5-1 2
Q15-6-1 2
Q16-1-1 3
Q16-2-1 3
Q16-3-1 3
Q16-4-1 3
Q16-5-1 3
Q16-6-1 3
Q16-1-2 9
Q16-2-2 9
Q16-3-2 

In [ ]:
# load jsonl file
path = f"dataset/test_data_random_{db}.jsonl"
with open(path, "r") as f:
    test_data = [json.loads(line) for line in f]

In [74]:
# test_data 'role': 'user'のcontentを取得
test_questions = []
for i, data in enumerate(test_data):
    user_question = data['messages'][1]['content']
    for question in questions:
        if question['user_question'] == user_question:
            test_questions.append(question)

id_list = []
for question in test_questions:
    if question['id'] not in id_list:
        id_list.append(question['id'])
    else:
        test_questions.remove(question)
    

In [75]:
len(test_questions)

54

In [76]:
print(test_questions[0]['user_question'])

Please tell me the ChEBI IDs of the compounds involved in the rhea:10024 reaction


In [79]:
from openai import OpenAI
client = OpenAI()

In [ ]:
system_prompt = "Create a SPARQL query to retrieve values from the database for the user question"

for q in test_questions:
  completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::Bif5J07q",
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": q["user_question"]}
    ]
  )
  q["llm_sparql"] = completion.choices[0].message.content

with open(save_path, 'w') as f:
    json.dump(test_questions, f, indent=4)

In [ ]:
import time
from functions.SPARQL_executer import execute_query

query_results = []
for question in test_questions:
    time.sleep(0.1)
    query_results += [execute_query(question, endpoint, "llm_sparql", 10000, "")]
    print(question["id"], len(query_results[-1][0]))

# 各質問に対する結果を追加
for result, question_id in query_results:
    # 質問を見つけて結果を追加
    for question in test_questions:
        if question["id"] == question_id:
            question["results"] = result

with open(save_path, 'w') as f:
    json.dump(test_questions, f, indent=4)

Q2-1-1 3
Q2-2-1 3
Q2-3-1 3
Q2-4-1 3
Q2-5-1 3
Q2-6-1 3
Q4-1-1 0
Q4-2-1 0
Q4-3-1 0
Q4-4-1 0
Q4-5-1 0
Q4-6-1 0
Q5-1-1 0
Q5-2-1 0
Q5-3-1 0
Q5-4-1 0
Q5-5-1 0
Q5-6-1 0
Q8-1-1 0
Q8-2-1 0
Q8-3-1 1
Q8-4-1 0
Q8-5-1 1
Q8-6-1 20
Q15-1-1 0
Q15-2-1 1
Q15-3-1 1
Q15-4-1 3
Q15-5-1 1
Q15-6-1 5
Q17-1-1 1
Q17-2-1 1
Q17-3-1 1
Q17-4-1 1
Q17-5-1 1
Q17-6-1 1
Q17-1-2 2
Q17-2-2 2
Q17-3-2 2
Q17-4-2 2
Q17-5-2 2
Q17-6-2 2
Q19-1-1 3
Q19-2-1 3
Q19-3-1 3
Q19-4-1 3
Q19-5-1 3
Q19-6-1 3
Q19-1-2 9
Q19-2-2 9
Q19-3-2 9
Q19-4-2 9
Q19-5-2 9
Q19-6-2 9


### evaluation

In [3]:
with open(save_path, 'r') as f:
    questions = json.load(f)

with open(save_path_with_results, 'r') as f:
    answers = json.load(f)

In [4]:
# questionsのもつidとanswersのもつidが一致するものを取得, answerを間引く
new_answers = []

for question in questions:
    for answer in answers:
        if question['id'] == answer['id']:
            new_answers.append(answer)


In [5]:
len(questions), len(new_answers)

(30, 30)

In [ ]:
print(questions[0]['llm_sparql'])

PREFIX genex: <http://purl.org/genex#>
PREFIX efo: <http://www.ebi.ac.uk/efo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX core: <http://purl.uniprot.org/core/>
PREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>

SELECT DISTINCT ?expression_condition_developmental_stage ?expression_level ?gene_alias 
WHERE {
    VALUES ?gene_name { "RGMA" }
    VALUES ?expression_condition_developmental_stage { efo:developmental_stage_uninformed efo:developmental_stage_embryonic efo:developmental_stage_fetal efo:developmental_stage_child efo:developmental_stage_adult }
    ?Expression a genex:Expression ;
        genex:hasExpressionCondition / genex:hasDevelopmentalStage / rdfs:label ?expression_condition_developmental_stage ;
        genex:hasExpressionLevel ?expression_level ;
        genex:hasSequenceUnit / lscr:xrefUniprot / core:recommendedName / core:fullName ?gene_name ;
        genex:hasSequenceUnit /

In [7]:
score = evaluate_jaccard(questions, new_answers)

Evaluating: 100%|██████████| 30/30 [00:00<00:00, 562.37it/s]

Skipping Q2-1-1 due to empty columns.
Skipping Q2-2-1 due to empty columns.
Skipping Q2-3-1 due to missing results.
Skipping Q2-4-1 due to empty columns.
Skipping Q2-5-1 due to empty columns.
Skipping Q2-6-1 due to missing results.
Skipping Q4-1-1 due to empty columns.
Skipping Q4-2-1 due to empty columns.
Skipping Q4-3-1 due to empty columns.
Skipping Q4-4-1 due to empty columns.
Skipping Q4-5-1 due to empty columns.
Skipping Q4-6-1 due to empty columns.
Skipping Q5-1-1 due to empty columns.
Skipping Q5-2-1 due to missing results.
Skipping Q5-3-1 due to empty columns.
Skipping Q5-4-1 due to empty columns.
Skipping Q5-5-1 due to empty columns.
Skipping Q5-6-1 due to empty columns.
Skipping Q7-1-1 due to empty columns.
Skipping Q7-2-1 due to missing results.
Skipping Q7-3-1 due to missing results.
Skipping Q7-4-1 due to empty columns.
Skipping Q7-5-1 due to empty columns.
Skipping Q7-6-1 due to empty columns.
Skipping Q16-1-1 due to missing results.
Skipping Q16-2-1 due to empty columns

In [9]:
score

{'Q2-1-1': {'jaccard_score': 0},
 'Q2-2-1': {'jaccard_score': 0},
 'Q2-3-1': {'jaccard_score': 0},
 'Q2-4-1': {'jaccard_score': 0},
 'Q2-5-1': {'jaccard_score': 0},
 'Q2-6-1': {'jaccard_score': 0},
 'Q4-1-1': {'jaccard_score': 0},
 'Q4-2-1': {'jaccard_score': 0},
 'Q4-3-1': {'jaccard_score': 0},
 'Q4-4-1': {'jaccard_score': 0},
 'Q4-5-1': {'jaccard_score': 0},
 'Q4-6-1': {'jaccard_score': 0},
 'Q5-1-1': {'jaccard_score': 0},
 'Q5-2-1': {'jaccard_score': 0},
 'Q5-3-1': {'jaccard_score': 0},
 'Q5-4-1': {'jaccard_score': 0},
 'Q5-5-1': {'jaccard_score': 0},
 'Q5-6-1': {'jaccard_score': 0},
 'Q7-1-1': {'jaccard_score': 0},
 'Q7-2-1': {'jaccard_score': 0},
 'Q7-3-1': {'jaccard_score': 0},
 'Q7-4-1': {'jaccard_score': 0},
 'Q7-5-1': {'jaccard_score': 0},
 'Q7-6-1': {'jaccard_score': 0},
 'Q16-1-1': {'jaccard_score': 0},
 'Q16-2-1': {'jaccard_score': 0},
 'Q16-3-1': {'jaccard_score': 0},
 'Q16-4-1': {'jaccard_score': 0},
 'Q16-5-1': {'jaccard_score': 0},
 'Q16-6-1': {'jaccard_score': 0},
 'ov